In [1]:
!pip install git+https://github.com/huggingface/transformers.git
!pip install git+https://github.com/huggingface/accelerate.git
!pip install -U datasets peft trl bitsandbytes


  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-bw4a6l6c
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-bw4a6l6c
  Resolved https://github.com/huggingface/transformers.git to commit 9300728665aaeb0ebf4db99f9d9fbce916b4a183
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.53.0.dev0-py3-none-any.whl size=11431110 sha256=752c322fc064e1dd7b9b5d230718a215d91d2655d8bf6a6a99abfadb3017e9e2
  Stored in directory: /tmp/pip-ephem-wheel-cache-jkhxe433/wheels/32/4b/78/f195c684dd3a9ed21f3b39fe8f85b48df7918581b6437be143
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.51.3
    Uninstalling transformers-4.51.3:
      Successfully uninstalled transformers-4.51.3
  Cloning https://github.co

Model page: https://huggingface.co/meta-llama/Llama-3.2-1B

⚠️ If the generated code snippets do not work, please open an issue on either the [model repo](https://huggingface.co/meta-llama/Llama-3.2-1B)
			and/or on [huggingface.js](https://github.com/huggingface/huggingface.js/blob/main/packages/tasks/src/model-libraries-snippets.ts) 🙏

The model you are trying to use is gated. Please make sure you have access to it by visiting the model page.To run inference, either set HF_TOKEN in your environment variables or Secrets or run the following cell to login. 🤗

In [2]:
from huggingface_hub import notebook_login
notebook_login()

**Cấu hình model và tokenizer**

In [3]:
import json
import torch
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from trl import SFTTrainer, SFTConfig
from peft import LoraConfig, get_peft_model

# Cấu hình quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

# Tải mô hình và tokenizer
base_model_name = "meta-llama/Llama-3.2-1B"
base_model = AutoModelForCausalLM.from_pretrained(
base_model_name,
quantization_config=bnb_config,
device_map={"": 0},
trust_remote_code=True,
torch_dtype=torch.bfloat16  # Ensure fp16
)
base_model.config.use_cache = False
base_model.config.pretraining_tp = 1

tokenizer = AutoTokenizer.from_pretrained(base_model_name, use_fast=True)
EOS_TOKEN = tokenizer.eos_token # do not forget this part!
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"  # Ensure right padding for causal LM

2025-06-16 09:18:03.636420: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750065483.847414      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750065483.910362      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

**LOAD DATASET**

In [5]:
#prompt
alpaca_prompt = """This is a question about cybersecurity. Please write an answer.

### Question:
{}

### Response:
{}"""
# Formatting function for LLaMA 3.2 chat template

def formatting_prompts_func(examples):
    questions = examples["question"]
    # inputs       = examples["input"]
    outputs      = examples["response"]
    texts = []
    for question, output in zip(questions, outputs):
            # if not response.endswith(tokenizer.eos_token):
            # response += tokenizer.eos_token
        text = alpaca_prompt.format(question, output) + EOS_TOKEN # without this token generation goes on forever!
        texts.append(text)
    return { "text" : texts, }

# Load dataset
dataset = load_dataset('json', data_files='/kaggle/input/cyber-security/hihihihi_cleaned.json', split="train")
dataset = dataset.map(formatting_prompts_func, batched = True)
print(dataset["text"][0])

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/718 [00:00<?, ? examples/s]

This is a question about cybersecurity. Please write an answer.

### Question:
Which of these devices would not be considered part of the Internet of Things?
A. Smartphone
B. Thermostat
C. Light bulb
D. Set-top cable box

### Response:
A. A thermostat is an embedded device without a traditional user interface. A light bulb
would have no user interface, even if it has network capabilities. A set-top cable box
would have a custom interface and not a general-purpose one. The only device here that
is a general-purpose computing platform with a traditional user interface—screen and
keyboard—is the smartphone, so it isn’t part of the IoT.<|end_of_text|>


**MODEL INFERENCE BEFORE FINETUNE**

In [6]:
inputs = tokenizer(
[
    alpaca_prompt.format(
        "What is the following command used for? sqlmap.py-u,,http://10.10.1.20/?p=1 &forumaction=search -dbs\nA.  Creating backdoors using SQL injection\nB.  A Enumerating the databases in the DBMS for the URL\nC.  Retrieving SQL statements being executed on the database\nD.  Searching database statements at the IP address given", # question
        "", #  - leave this blank for generation!
    ) + tokenizer.eos_token
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = base_model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|>This is a question about cybersecurity. Please write an answer.

### Question:
What is the following command used for? sqlmap.py-u,,http://10.10.1.20/?p=1 &forumaction=search -dbs
A.  Creating backdoors using SQL injection
B.  A Enumerating the databases in the DBMS for the URL
C.  Retrieving SQL statements being executed on the database
D.  Searching database statements at the IP address given

### Response:
<|end_of_text|><|begin_of_text|>Question: You are given a URL that contains a SQL injection in the query string. You can use the sqlmap tool to enumerate all the databases that exist in the database server. In the next step, you can use the backdoor that sqlmap tool to create a backdoor for the given database. The backdoor will be used to retrieve the SQL statements that are executed on the database.

sqlmap.py-u,http://10.10.1.20/?p=1 &forumaction=search -dbs
You can use the backdoor that sqlmap tool to create a backdoor for the given database. The back


In [11]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer
import torch

# Define the alpaca prompt (same as provided)
alpaca_prompt = """This is a question about cybersecurity. Please write an answer.

### Question:
{}

### Response:
{}"""

# Load the CSV file
csv_file = "/kaggle/input/cyber-security/B1.csv"  # Path to the input CSV
df = pd.read_csv(csv_file)

# Initialize TextStreamer
text_streamer = TextStreamer(tokenizer)

# List to store predictions
predictions = []

# Iterate through each row in the CSV
for index, row in df.iterrows():
    question = row['question']
    ground_truth = row['ground_truth']
    
    # Create prompt for the question
    prompt = alpaca_prompt.format(question, "") + tokenizer.eos_token
    
    # Tokenize input
    inputs = tokenizer(
        [prompt],
        return_tensors="pt"
    ).to("cuda")
    
    # Generate answer from the pre-trained model
    with torch.no_grad():  # Disable gradients to save resources
        outputs = base_model.generate(
            **inputs,
            # streamer=text_streamer,  # Comment out to avoid streaming during batch processing
            max_new_tokens=200,
            eos_token_id=tokenizer.eos_token_id
        )
    
    # Decode output
    response = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
    
    # Extract the answer from the response
    if "### Response:" in response:
        predicted_response = response.split("### Response:")[-1].strip()
    else:
        predicted_response = response.strip()  # Fallback if ### Response: is missing
    
    # Optionally standardize predicted_response (e.g., extract A, B, C, or D)
    # Uncomment the following line if you want to ensure the answer is a single letter (A, B, C, D)
    # predicted_response = predicted_response[0] if predicted_response and predicted_response[0] in ['A', 'B', 'C', 'D'] else predicted_response
    
    # Store the result
    predictions.append({
        'index': row['index'],
        'question': question,
        'llm_answer': predicted_response,
        'ground_truth': ground_truth
    })

# Save predictions to a CSV file
results_df = pd.DataFrame(predictions)
output_csv = "/kaggle/working/test_result_prefinetune.csv"
results_df.to_csv(output_csv, index=False, encoding='utf-8')
print(f"Results saved to {output_csv}")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for

Results saved to /kaggle/working/test_result_prefinetune.csv


**FineTune**

In [12]:
# Cấu hình LoRA
# LoRA config
peft_config = LoraConfig(
    lora_alpha=16,                          
    lora_dropout=0.05,                       
    r=64,                                    
    bias="none",                             
    task_type="CAUSAL_LM",                  
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],  
)

base_model = get_peft_model(base_model, peft_config)

# Cấu hình SFTConfig
training_args = SFTConfig(
output_dir="./results",
per_device_train_batch_size=1,
gradient_accumulation_steps=2, 
warmup_steps=5,
num_train_epochs=1,
learning_rate=2e-4,
fp16=False,  
bf16=False,  
logging_steps=0.2,
optim="adamw_8bit",
weight_decay=0.01,
lr_scheduler_type="linear",
seed=3407,
max_seq_length=512,
  # Use formatted text field
report_to="none" # Disable logging
)

from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

# Khởi tạo SFTTrainer
trainer = SFTTrainer(
model=base_model,
train_dataset=dataset,
peft_config=peft_config,
args=training_args,
data_collator = data_collator
   # Suppress warning
)

Converting train dataset to ChatML:   0%|          | 0/718 [00:00<?, ? examples/s]

Adding EOS to train dataset:   0%|          | 0/718 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/718 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/718 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [13]:

trainer.train()

Step,Training Loss
36,1.110100
72,1.076300
108,0.924300
144,0.960900
180,0.986700


TrainOutput(global_step=180, training_loss=1.011658689710829, metrics={'train_runtime': 381.2178, 'train_samples_per_second': 1.883, 'train_steps_per_second': 0.472, 'total_flos': 1468701990912000.0, 'train_loss': 1.011658689710829})

**MODEL AFTER FINETUNE**

In [14]:
inputs = tokenizer(
[
    alpaca_prompt.format(
        "What is the following command used for? sqlmap.py-u,,http://10.10.1.20/?p=1 &forumaction=search -dbs\nA.  Creating backdoors using SQL injection\nB.  A Enumerating the databases in the DBMS for the URL\nC.  Retrieving SQL statements being executed on the database\nD.  Searching database statements at the IP address given", # question
        "", #  - leave this blank for generation!
    ) + tokenizer.eos_token
], return_tensors = "pt").to("cuda")

# outputs = base_model.generate(
#     input_ids=inputs.input_ids,
#     attention_mask=inputs.attention_mask,
#     max_new_tokens=128,
#     eos_token_id=tokenizer.eos_token_id,
#     use_cache=True,
# )
# response = tokenizer.batch_decode(outputs, skip_special_tokens=True)
# print(response)
from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = base_model.generate(**inputs, streamer = text_streamer, max_new_tokens = 200)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|>This is a question about cybersecurity. Please write an answer.

### Question:
What is the following command used for? sqlmap.py-u,,http://10.10.1.20/?p=1 &forumaction=search -dbs
A.  Creating backdoors using SQL injection
B.  A Enumerating the databases in the DBMS for the URL
C.  Retrieving SQL statements being executed on the database
D.  Searching database statements at the IP address given

### Response:
<|end_of_text|><|begin_of_text|>#1. A, B, D. The -dbs option is used to enumerate the databases in the DBMS. The -u option is used to create a backdoor, and the -h is used to specify a host. The -dbs option is used to search the database for SQL statements. The -d option is used to specify the database to search. The -u option is used to specify a username. The -h is used to specify a host. The -p option is used to specify a password. The -d option is used to specify the database to search. The -u option is used to specify a username. The -h option is used to spec

In [15]:
import pandas as pd
from transformers import TextStreamer
import torch

# Định nghĩa alpaca_prompt gốc
alpaca_prompt = """This is a question about cybersecurity. Please write an answer.

### Question:
{}

### Response:
{}"""

# Đọc file CSV
csv_file = "/kaggle/input/cyber-security/B1.csv"  # Thay bằng đường dẫn thực tế
df = pd.read_csv(csv_file)
# Khởi tạo TextStreamer
text_streamer = TextStreamer(tokenizer)

# Danh sách để lưu kết quả dự đoán
predictions = []

# Duyệt qua từng dòng trong file CSV
for index, row in df.iterrows():
    question = row['question']
    ground_truth = row['ground_truth']
    
    # Tạo prompt cho câu hỏi
    prompt = alpaca_prompt.format(question, "") + tokenizer.eos_token
    
    # Tokenize input giống như trong mã gốc
    inputs = tokenizer(
        [prompt],
        return_tensors="pt"
    ).to("cuda")
    
    # Sinh đáp án từ mô hình, sử dụng TextStreamer
    with torch.no_grad():  # Tắt gradient để tiết kiệm tài nguyên
        outputs = base_model.generate(
            **inputs,
            # streamer=text_streamer,
            max_new_tokens=200,
            eos_token_id=tokenizer.eos_token_id
        )
    
    # Giải mã output
    response = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
    
    # Trích xuất phần đáp án từ response
    if "### Response:" in response:
        predicted_response = response.split("### Response:")[-1].strip()
    else:
        predicted_response = response.strip()  # Nếu không có ### Response:, lấy toàn bộ output
    
    # Chuẩn hóa predicted_response để so sánh (giả định đáp án là A, B, C, hoặc D)
    # predicted_response = predicted_response[0] if predicted_response and predicted_response[0] in ['A', 'B', 'C', 'D'] else predicted_response
    
    # Lưu kết quả
    predictions.append({
        'index': row['index'],
        'question': question,
        'llm_answer': predicted_response,
        'ground_truth': ground_truth
    })
    


# Lưu kết quả dự đoán vào file CSV
results_df = pd.DataFrame(predictions)
results_df.to_csv("/kaggle/working/result_finetune.csv", index=False, encoding='utf-8')



Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for

**SAVE MODEL**

In [16]:
new_model_name = "Test_AI_security"
trainer.model.push_to_hub(new_model_name)
tokenizer.push_to_hub(new_model_name)
trainer.processing_class.push_to_hub(new_model_name)

Uploading...:   0%|          | 0.00/180M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

Uploading...:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

Uploading...:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/sypher47/Test_AI_security/commit/6a66487c75be32ef6ae7a2385ec677d00b923912', commit_message='Upload tokenizer', commit_description='', oid='6a66487c75be32ef6ae7a2385ec677d00b923912', pr_url=None, repo_url=RepoUrl('https://huggingface.co/sypher47/Test_AI_security', endpoint='https://huggingface.co', repo_type='model', repo_id='sypher47/Test_AI_security'), pr_revision=None, pr_num=None)

**Chạy thử model đã lưu**

In [17]:
del base_model
del trainer
torch.cuda.empty_cache()
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel
import torch

# Base model
base_model_id = "meta-llama/Llama-3.2-1B"

# Your fine-tuned LoRA adapter repository
lora_adapter_id = 'sypher47/Test_AI_security'

# Load the model in 4-bit
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

# Load base model
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    quantization_config=bnb_config,
    trust_remote_code=True,
)

# Attach the LoRA adapter
model = PeftModel.from_pretrained(
    base_model,
    lora_adapter_id,
    device_map="auto",
    trust_remote_code=True,
)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model_id, trust_remote_code=True)

adapter_config.json:   0%|          | 0.00/856 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/180M [00:00<?, ?B/s]

In [18]:
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Tony wants to integrate a 128-bit symmetric block cipher with key sizes of 128,192, or 256 bits into a software program, which involves 32 rounds of computational operations that include substitution and permutation operations on four 32-bit word blocks using 8-variable S-boxes with 4-bit entry and 4-bit exit. Which of the following algorithms includes all the above features and can be integrated by Tony into the software program?\nA. TEA\nB. CAST-128\nC. RC5\nD. serpent", # input
        "", # output - leave this blank for generation!
    ) + tokenizer.eos_token
], return_tensors = "pt").to("cuda")

# outputs = base_model.generate(
#     input_ids=inputs.input_ids,
#     attention_mask=inputs.attention_mask,
#     max_new_tokens=280,
#     eos_token_id=tokenizer.eos_token_id,
#     use_cache=True,
# )
from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 150)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|>This is a question about cybersecurity. Please write an answer.

### Question:
Tony wants to integrate a 128-bit symmetric block cipher with key sizes of 128,192, or 256 bits into a software program, which involves 32 rounds of computational operations that include substitution and permutation operations on four 32-bit word blocks using 8-variable S-boxes with 4-bit entry and 4-bit exit. Which of the following algorithms includes all the above features and can be integrated by Tony into the software program?
A. TEA
B. CAST-128
C. RC5
D. serpent

### Response:
<|end_of_text|><|begin_of_text|># C. The CAST-128 algorithm is a block cipher algorithm that is a modification of the RC5 algorithm. It was created by a team of cryptographers in 1997. The algorithm is a 128-bit block cipher algorithm that uses a 128-bit key, 32 rounds of substitution and permutation operations on 4 32-bit word blocks, and a 4-bit S-box. The algorithm uses a 4-bit S-box that is a permutation of a 